In [1]:
type MarkerMatrix
    X::Array{Float64,2}
    xArray::Array{Array{Float64,1},1}
    XpRinvX::Array{Float64,1}
    markerMeans::Array{Float64,2}
    mean2pq::Float64  
    centered::Bool
    function MarkerMatrix(X::Array{Float64,2})
        markerMeans = center!(X) #centering
        p           = markerMeans/2.0
        mean2pq     = (2*p*(1-p)')[1,1]
        xArray = get_column_ref(X)
        XpRinvX = getXpRinvX(X)
        new(X,xArray,XpRinvX,markerMeans,mean2pq,true)
    end
end

In [ ]:
m=MarkerMatrix()

In [3]:
function get_column(X,j)
    nrow,ncol = size(X)
    if j>ncol||j<0
        error("column number is wrong!")
    end
    indx = 1 + (j-1)*nrow
    ptr = pointer(X,indx)
    pointer_to_array(ptr,nrow)
end

function get_column_ref(X)
    ncol = size(X)[2]
    xArray = Array(Array{Float64,1},ncol)
    for i=1:ncol
        xArray[i] = get_column(X,i)
    end
    return xArray
end

function center!(X)
    nrow,ncol = size(X)
    colMeans = mean(X,1)
    BLAS.axpy!(-1,ones(nrow)*colMeans,X)
    return colMeans
end

function getXpRinvX(X)
    ncol = size(X)[2]
    XpRinvX = [((X[:,i])'X[:,i])[1]::Float64 for i=1:ncol]
    return XpRinvX
end

getXpRinvX (generic function with 1 method)

In [4]:
a=randn(5,10)

5x10 Array{Float64,2}:
  0.58319   -0.323728   0.375674  …  -1.47162    0.0591578  0.100419
  1.04847   -0.41908    0.168393     -1.1842    -0.0808352  0.254844
  0.278464   0.292039   0.344062     -0.667329   1.61426    0.120778
 -2.13791   -0.920049  -0.954589     -0.732919   1.91075    0.428817
  0.583479   1.01493   -0.069208     -0.209014  -0.283806   1.65456 

In [7]:
X=copy(a)     
markerMeans = center!(X) #centering
        p           = markerMeans/2.0
        mean2pq     = (2*p*(1-p)')[1,1]
        xArray = get_column_ref(X)
        XpRinvX = getXpRinvX(X)


10-element Array{Float64,1}:
  0.84599
  3.39994
 13.3595 
  8.7958 
 10.2434 
  3.51132
  2.60163
  7.6279 
  9.21536
  3.99036

In [5]:
MarkerMatrix(a)

MarkerMatrix(5x10 Array{Float64,2}:
  0.51205   -0.25255    0.402807   …  -0.618602  -0.584749  -0.411466 
  0.977331  -0.347902   0.195527      -0.331181  -0.724741  -0.25704  
  0.207324   0.363217   0.371196       0.185686   0.970354  -0.391107 
 -2.20905   -0.848871  -0.927456       0.120096   1.26685   -0.0830676
  0.51234    1.08611   -0.0420743      0.644001  -0.927712   1.14268  ,[[0.5120504318116091,0.9773313713593385,0.20732372838538982,-2.2090451313593062,0.5123395998029686],[-0.25255003641602136,-0.34790188458188254,0.36321734481245604,-0.8488706592636441,1.086105235449092],[0.40280732810570696,0.19552671846544606,0.371195845361449,-0.9274556406365656,-0.04207425129603637],[-2.737881620787373,-0.44443090712487876,0.5272645043352661,0.5045196828191529,2.1505283407578326],[0.10589781829075032,0.4601224261876191,0.17174937085760042,0.5914852623945341,-1.329254877730504],[-0.2061978944165538,-1.1646262988820069,-0.502319150891037,1.3298534482834343,0.5432898959061633],[0.196494

In [12]:
typeof(a)

Bool

In [8]:
type hao
    speed::Float64
end

In [15]:
methods(Signed)

2-element Array{Any,1}:
 call{T}(::Type{T}, arg) at essentials.jl:56    
 call{T}(::Type{T}, args...) at essentials.jl:57

In [17]:
include("src/JWAS.jl")

JWAS

In [18]:
using JWAS

In [20]:
using(Distributions)
d = Binomial(2,0.5)

nObs     = 10
nMarkers = 100
X        = float(rand(d,(nObs,nMarkers)))
α        = randn(nMarkers)
a        = X*α
stdGen   = std(a)
a        = a/stdGen
y        = a + randn(nObs);

In [21]:
myOption=Dict()
myOption["run"]           = "BayesC"
myOption["seed"]          = 314
myOption["chainLength"]   = 5000
myOption["probFixed"]     = 0.95
myOption["estimatePi"]    = "yes"
myOption["estimateScale"] = "yes"
myOption["varGenotypic"]  = 1
myOption["varResidual"]   = 1

output = runJWAS(myOption,X,y)

This is iteration 100, number of loci 0, vara 0.0, vare 0.5404901221698128
This is iteration 200, number of loci 2, vara 0.18297887876090804, vare 0.5466295405439487
This is iteration 300, number of loci 9, vara 1.0110326920086825, vare 0.28324694552241586
This is iteration 400, number of loci 13, vara 0.43658368035756406, vare 1.243910791446036
This is iteration 500, number of loci 24, vara 1.040367433835445, vare 0.6485584614726785
This is iteration 600, number of loci 16, vara 0.12539624954599782, vare 0.30320621532691666
This is iteration 700, number of loci 54, vara 1.0330473493431744, vare 1.0590426205094001
This is iteration 800, number of loci 1, vara 1.6318089015045685, vare 0.5979360956974316
This is iteration 900, number of loci 5, vara 0.09301259169011483, vare 0.946619633714233
This is iteration 1000, number of loci 1, vara 0.16245699861366242, vare 0.8640385255354203
This is iteration 1100, number of loci 52, vara 0.8527548761848736, vare 0.4253928159651683
This is iterat

Dict{Any,Any} with 7 entries:
  "posterior sample of ge… => [0.217815,0.484239,0.146367,0.650997,1.04043,0.36…
  "posterior mean of fixe… => [-0.413069904269796]
  "posterior mean of mark… => [-0.0120968,0.023964,0.000594045,0.00896536,-0.00…
  "posterior sample of pi" => [0.965886,0.9536,0.934067,0.869616,0.879326,0.905…
  "posterior sample of re… => [0.608055,0.612593,0.439942,0.328555,0.398215,0.3…
  "posterior sample of sc… => [0.203768,0.132091,0.0339523,0.255402,0.137085,0.…
  "model frequency"        => [0.193,0.201,0.1724,0.1844,0.196,0.1986,0.2002,0.…

, number of loci 2, vara 0.3727843059089751, vare 0.9187865138483945
This is iteration 4700, number of loci 4, vara 0.655581122645697, vare 1.6511840556340718
This is iteration 4800, number of loci 54, vara 1.2375437685702997, vare 0.43591078683133805
This is iteration 4900, number of loci 67, vara 0.8464970573142764, vare 0.5165215770897236
This is iteration 5000, number of loci 58, vara 0.22611717656182398, vare 0.7953725742456512


In [13]:
type OutputValues1
    meanFxdEff::Float64 
    meanMrkEff::Float64
    mdlFrq::Float64
    resVar::Float64
    genVar::Float64
    pi::Float64
    scale::Float64
`end

In [14]:
a=OutputValues1()

OutputValues1(0.0,0.0,0.0,0.0,0.0,0.0,0.0)

In [8]:
a

OutputValues